# **Customer Lifetime Value (CLV) & Churn Prediction**

## **Problem Statement** : 
In the highly competitive non-contractual retail sector, customer acquisition costs are rising while customer loyalty is declining. This project addresses the challenge of ***Silent Attrition***—where customers simply stop buying without a formal cancellation. By leveraging transactional data, this project builds a predictive engine to identify high-value customers at risk of churning, enabling the business to proactively allocate marketing resources where they yield the highest ROI.

### **Objectives**
* Predict which customers are likely to churn in the next 6–12 months
* Estimate how much revenue each customer will generate in the future (CLV)
* Provide insights to retain high-value, high-risk customers
* Allow business managers to test strategies using a What-If simulation dashboard

### **Phases**
I will be executing this project in 7 phases in a structured way:
* **Phase 1 :** Data Cleaning & Transactional 
* **Phase 2 :** EDA - Exploratory Data Analysis
* **Phase 3 :** Feature Engineering - The RFM Transformation.
* **Phase 4 :** Probabilistic CLV Modeling - Predict future spending without using standard regression.
* **Phase 5 :** Churn Prediction
* **Phase 6 :** Dashboard with Discount vs Churn Impact
* **Phase 7:** Business Insights & Strategy

***Key Business Metrics***

* **CLV**                - Future revenue expected from a customer
* **Churn Probability**  - Likelihood that a customer will stop buying
* **Revenue at Risk**    - CLV × Churn probability

In [90]:
# First we need to import all the necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

from lifetimes.utils import summary_data_from_transaction_data, calibration_and_holdout_data
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.plotting import plot_period_transactions

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, mean_absolute_error, mean_squared_error

In [91]:
# Loading the dataset
df = pd.read_csv('online_retail_II.csv')
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [92]:
# Checking shape of the dataset
df.shape

(1067371, 8)

Our dataset has more than 10 lakhs of rows and 8 columns

In [93]:
# Explore Data Structure
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


### ***Phase 1 :*** Data Cleaning & Transactional Logic
* Actions:
    * Standardizing columns name
    * Correcting Datatype
    * Removing rows with missing values.
    * Removing Duplicate orders.
    * Handling Returned/Cancelled orders.
    * Creating a '**total price**' (Quantity*Price) column.

First thing to do is make sure each column name is clean and standardize for avoiding any future complications.

In [94]:
df.columns = df.columns.str.lower().str.replace(" ", "_")
df.columns


Index(['invoice', 'stockcode', 'description', 'quantity', 'invoicedate',
       'price', 'customer_id', 'country'],
      dtype='object')

The datatype of 'InvoiceDate' column is object. That's needed to be change into datettime datatype.

In [95]:
# Converting 'InvoiceDate' to datetime format and removing rows with invalid dates
df['invoice_date'] = pd.to_datetime(df['invoicedate'])
df.drop(columns=['invoicedate'], inplace=True)
df = df[df['invoice_date'].notnull()]

In [96]:
# Checking for missing values
df.isnull().sum()

invoice              0
stockcode            0
description       4382
quantity             0
price                0
customer_id     243007
country              0
invoice_date         0
dtype: int64

In [97]:
# Calculating percentage of missing values
missing = df.isnull().mean().round(3)*100
print("Missing values (%):\n", missing)

Missing values (%):
 invoice          0.0
stockcode        0.0
description      0.4
quantity         0.0
price            0.0
customer_id     22.8
country          0.0
invoice_date     0.0
dtype: float64


Customer ID has 22.8% of missing values (around 2.5 lacs). We need to drop these rows since customer ID is crucial for CLV modelling.

In [98]:
df.dropna(subset = ['customer_id'], inplace=True)

In [99]:
# Checking for missing values
df.isnull().sum()

invoice         0
stockcode       0
description     0
quantity        0
price           0
customer_id     0
country         0
invoice_date    0
dtype: int64

Now, there no missing values

In [100]:
# Checking for duplicate entries
df.duplicated().sum()

26479

There are 26,749 duplicate rows which need to drop.

In [101]:
# Removing duplicate rows
df = df.drop_duplicates()

In [102]:
# Checking for shape after removing missing values
df.shape

(797885, 8)

Now I am left with 7,49,885 rows which are enough for model building.

In [103]:
# Let's also check for the date range of the dataset
print(df['invoice_date'].min(),' to ', df['invoice_date'].max())

2009-12-01 07:45:00  to  2011-12-09 12:50:00


Our dataset ranges from 1 Dec 2009 to 09 Dec 2011.

I also need to create a column as '*Total Price*' which shows the total amount spent by the customer on that order.

In [104]:
df['total_price'] = df['quantity'] * df['price']
df.head()

,invoice,stockcode,description,quantity,price,customer_id,country,invoice_date,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,2009-12-01 07:45:00,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01 07:45:00,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01 07:45:00,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,2009-12-01 07:45:00,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,2009-12-01 07:45:00,30.0


In [105]:
df.describe()

,quantity,price,customer_id,invoice_date,total_price
count,797885.000000,797885.000000,797885.000000,797885,797885.000000
mean,12.602980,3.702732,15313.062777,2011-01-02 13:17:34.141160704,20.416465
min,-80995.000000,0.000000,12346.000000,2009-12-01 07:45:00,-168469.600000
25%,2.000000,1.250000,13964.000000,2010-07-02 09:47:00,4.350000
50%,5.000000,1.950000,15228.000000,2010-12-02 12:33:00,11.700000
75%,12.000000,3.750000,16788.000000,2011-07-31 15:50:00,19.500000
max,80995.000000,38970.000000,18287.000000,2011-12-09 12:50:00,168469.600000
std,191.670371,71.392549,1696.466663,NaN,313.518824


Seems like there are few negative values in the 'Quantity' column. Could be the returns order. Since these returned orders can affect the CLV model, I am creating a new dataframe to store these returned orders and removing these orders from the origional dataset.

In [106]:
returns_df = df[df['quantity'] < 0] # separate returns 
df = df[df['quantity'] > 0] # keep only purchases

In [107]:
# Final look at the cleaned dataset
df.head()

,invoice,stockcode,description,quantity,price,customer_id,country,invoice_date,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,6.95,13085.0,United Kingdom,2009-12-01 07:45:00,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01 07:45:00,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,6.75,13085.0,United Kingdom,2009-12-01 07:45:00,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2.10,13085.0,United Kingdom,2009-12-01 07:45:00,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.25,13085.0,United Kingdom,2009-12-01 07:45:00,30.0


Now, the Dataset is cleaned from all the null and duplicate values, returned orders and correct datatypes. With this, I can move forward with the phase 2.

In [108]:
# Saving the cleaned dataset
df.to_csv("cleaned_transactions.csv", index=False)
print("File Saved!")


File Saved!


In [109]:
# def preprocess_data(df):
#     df = df.copy()
#     df.columns = df.columns.str.lower().str.replace(" ", "_")
#     df = df.drop_duplicates()
#     df = df[df['customer_id'].notnull()]
#     df['invoice_date'] = pd.to_datetime(df['invoice_date'], errors='coerce')
#     df = df[df['invoice_date'].notnull()]
#     df['total_price'] = df['quantity'] * df['price']
#     df['is_return'] = df['quantity'] < 0
#     df = df[df['is_return'] == False]
#     return df
# cleaned_df = preprocess_data(df)